Install Ollama 

In [ ]:

!curl -fsSL https://ollama.com/install.sh | sh


Using llama3 and nomic-embed-text


In [1]:
!ollama serve & ollama pull mistral-nemo & ollama pull nomic-embed-text

Error: listen tcp 127.0.0.1:11434: bind: address already in use
pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest 
pulling manifest 
pulling b559938ab7a0... 100% ▕████████████████▏ 7.1 GB                         
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling f023d1ce0e55... 100% ▕████████████████▏  688 B                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         pul

In [2]:
!pip -qq install langchain
!pip -qq install langchain-core
!pip -qq install langchain-community

In [3]:
!ollama run mistral-nemo

⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠏ ⠙ ⠹ ⠸ ⠼ ⠼ ⠦ ⠧ ⠧ >>> Send a message (/? for help)
Use Ctrl + d or /bye to exit.
>>> Send a message (/? for help)
>>> Send a message (/? for help)

In [4]:
!pip install ollama langchain beautifulsoup4 chromadb gradio pymupdf -q

In [2]:
import gradio as gr
import ollama
from bs4 import BeautifulSoup as bs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import fitz 

/Users/gregoriokoslinskineto/miniconda3/envs/mestrado/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


Using PDF

In [4]:
# Function to read text from a PDF
def read_pdf(file_path):
    pdf_text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            pdf_text += page.get_text()
    return pdf_text

# Load the data from a PDF file
pdf_path = "krebs_casiopacheco_m.pdf"  # Change to the path of your PDF
pdf_content = read_pdf(pdf_path)


# Split the loaded PDF content into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.create_documents([pdf_content])


Using URL

In [ ]:
# Load the data from the web URL
url='https://pt.wikipedia.org/wiki/Resident_Evil_4'
loader = WebBaseLoader(url)
docs = loader.load()

# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

RAG Flow

In [5]:
# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Define the function to call the Ollama Mistral-nemo model
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='mistral-nemo', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Define the RAG setup
retriever = vectorstore.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return ollama_llm(question, formatted_context)




/var/folders/h0/0dsskf1n0ts_874pj62szfsm0000gn/T/ipykernel_10356/1797064749.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")


Interface

In [6]:
# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)

# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
  outputs="text",
  title="RAG with Mistral-nemo",
  description="Ask questions about the proveded context",
)

# Launch the Gradio app
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
